Мы заметили, что забыли удалить стоп-слова, когда строили прогностические модели для предсказания наличия в описаниях фильмов криминального сюжета. Стоит заметить, что это особо не повлияло на качество моделей, но мы всё же решили реализовать анализ с удалением стоп-слов. Чтобы было интереснее, мы выбрали другую тему для разметки. 
Достаточно часто в мелодрамах действие происходит в небольших городах или героини являются провинциалками. Мы закодировали тексты, содержащие упоминание о селе, деревне, провинциальном происхождении героев как 2, в противном случае ставили 1. Предварительно тексты, как и в предыдущем анализе, были отсортированы в случайном порядке, было размечено 490 описаний фильмов (85% от всей совокупности). 

In [1]:
import pandas as pd
df = pd.read_excel ('выгрузка_село.xlsx', index_col = 0)
df.head()

,title,year,description,desc_bin,province
1,Он – дракон,2015,"В давние времена не было у людей счастья, вмес...",1,1.0
2,Спасти мужа,2011,семья Сергея и Ольги Князевых. Однажды бизнес ...,0,1.0
3,Кружева,2014,"о талантливой девушке Вере, которая работает ...",0,2.0
4,Пассажир из Сан-Франциско,2019,Калифорния. 1 сентября 2004 года. По шоссе вдо...,1,1.0
5,Красавец и чудовище,2014,В 2016 году Вероника Пляшкевич и Алексей Анище...,1,1.0


Смотрим на наполненность категорий и замечаем, что в наиболее важной для нас категории (2 - фильм содержит упоминание села, деревни, провинциального происхождения героев) наблюдений меньше, чем в другой, более чем в 2 раза. Опять же мы решили не уменьшать кол-во наблюдений, закодированных единицей, по причине того, что сама база имеет небольшой размер – содержит 577 описаний фильмов. 

In [2]:
df['province'].value_counts()

1.0    367
2.0    123
Name: province, dtype: int64

Выполняем предобработку данных: удаляем небуквенные символы, приводим слова к одному регистру, удаляем двойные пробелы, латинские символы (вроде их и не было, но на всякий случай) и лемматизируем.

In [3]:
def delete_punctuation(text):
    new_text = []
    for i in text:
        if i.isalpha() or i == ' ':
            new_text.append(i)
        else:
            new_text.append(" ")
    return ''.join(new_text)

In [4]:
df['description_prep'] = df['description'].apply(delete_punctuation)
df.loc[4, 'description_prep']

'Калифорния    сентября      года  По шоссе вдоль побережья мчится автомобиль  По радио передают срочное сообщение из России о теракте в школе города Беслан  Автомобиль сворачивает на обочину  Водитель  Фил Маковский  выходит и  следуя инструкциям  которые ему передают по телефону  движется к уединенному пляжу  Там он обнаруживает свою дочь Машу  Девушка и ее приятель в состоянии наркотического опьянения  Рядом с молодыми людьми на утесе сидит наркодилер  Лысый   Фил набрасывается на него  тот сбрасывает Маковского в воду Пока Фил выбирается на берег  Лысому удается скрыться  его забирает подъехавший на мотоцикле сообщник  Маковский приводит дочь в чувство  берет на руки ее отключившегося приятеля и забирает обоих подростков  Сан Франциско       год  Во время прогулки по набережной Маковский смотрит на здание федеральной тюрьмы Алькатрас  Он рассуждает о том  что во время своей вынужденной эмиграции он не раз мог оказаться в ее стенах Начальные строки рассказа  Господин из Сан Франциск

In [5]:
df['description_prep'] = df['description_prep'].str.lower()
df.loc[4, 'description_prep']

'калифорния    сентября      года  по шоссе вдоль побережья мчится автомобиль  по радио передают срочное сообщение из россии о теракте в школе города беслан  автомобиль сворачивает на обочину  водитель  фил маковский  выходит и  следуя инструкциям  которые ему передают по телефону  движется к уединенному пляжу  там он обнаруживает свою дочь машу  девушка и ее приятель в состоянии наркотического опьянения  рядом с молодыми людьми на утесе сидит наркодилер  лысый   фил набрасывается на него  тот сбрасывает маковского в воду пока фил выбирается на берег  лысому удается скрыться  его забирает подъехавший на мотоцикле сообщник  маковский приводит дочь в чувство  берет на руки ее отключившегося приятеля и забирает обоих подростков  сан франциско       год  во время прогулки по набережной маковский смотрит на здание федеральной тюрьмы алькатрас  он рассуждает о том  что во время своей вынужденной эмиграции он не раз мог оказаться в ее стенах начальные строки рассказа  господин из сан франциск

In [6]:
df

,title,year,description,desc_bin,province,description_prep
1,Он – дракон,2015,"В давние времена не было у людей счастья, вмес...",1,1.0,в давние времена не было у людей счастья вмес...
2,Спасти мужа,2011,семья Сергея и Ольги Князевых. Однажды бизнес ...,0,1.0,семья сергея и ольги князевых однажды бизнес ...
3,Кружева,2014,"о талантливой девушке Вере, которая работает ...",0,2.0,о талантливой девушке вере которая работает ...
4,Пассажир из Сан-Франциско,2019,Калифорния. 1 сентября 2004 года. По шоссе вдо...,1,1.0,калифорния сентября года по шоссе вдо...
5,Красавец и чудовище,2014,В 2016 году Вероника Пляшкевич и Алексей Анище...,1,1.0,в году вероника пляшкевич и алексей анище...
...,...,...,...,...,...,...
573,Дом для куклы,2016,"\nГлавная героиня сериала, девушка по имени Ди...",0,NaN,главная героиня сериала девушка по имени дин...
574,Прилетит вдруг волшебник,2008,Вера - красивая молодая женщина никак не может...,0,NaN,вера красивая молодая женщина никак не может...
575,Вопреки всему,2014,"играет молодую девушку Наташу, жизнь которой ...",0,NaN,играет молодую девушку наташу жизнь которой ...
576,Хозяйка «Белых ночей»,2011,\nМарина – хозяйка уютной гостиницы «Белые ноч...,0,NaN,марина хозяйка уютной гостиницы белые ночи...


In [7]:
def delete_double_space(double_spaced_text):
    while '  ' in double_spaced_text:
        double_spaced_text = double_spaced_text.replace('  ', ' ')
    return double_spaced_text

In [8]:
df['description_prep'] = df['description_prep'].apply(delete_double_space)
df.loc[4, 'description_prep']

'калифорния сентября года по шоссе вдоль побережья мчится автомобиль по радио передают срочное сообщение из россии о теракте в школе города беслан автомобиль сворачивает на обочину водитель фил маковский выходит и следуя инструкциям которые ему передают по телефону движется к уединенному пляжу там он обнаруживает свою дочь машу девушка и ее приятель в состоянии наркотического опьянения рядом с молодыми людьми на утесе сидит наркодилер лысый фил набрасывается на него тот сбрасывает маковского в воду пока фил выбирается на берег лысому удается скрыться его забирает подъехавший на мотоцикле сообщник маковский приводит дочь в чувство берет на руки ее отключившегося приятеля и забирает обоих подростков сан франциско год во время прогулки по набережной маковский смотрит на здание федеральной тюрьмы алькатрас он рассуждает о том что во время своей вынужденной эмиграции он не раз мог оказаться в ее стенах начальные строки рассказа господин из сан франциско ивана бунина представляются маковском

In [9]:
import re
def delete_latin(text):    
    new_text = []
    for symbol in str(text):
        if symbol not in re.findall(r'[A-Za-z]', str(text)):
            new_text.append(symbol)
    return ''.join(new_text)

In [10]:
df['description_prep'] = df['description_prep'].apply(delete_latin)

In [11]:
import pymystem3
mstem = pymystem3.Mystem()

In [12]:
def lemmas(not_lemmatized_text):
    return ''.join(mstem.lemmatize(not_lemmatized_text)).strip()

In [13]:
%%time

df['description_prep'] = df['description_prep'].apply(lemmas)
df

Wall time: 13min 2s


,title,year,description,desc_bin,province,description_prep
1,Он – дракон,2015,"В давние времена не было у людей счастья, вмес...",1,1.0,в давний время не быть у человек счастие вмест...
2,Спасти мужа,2011,семья Сергея и Ольги Князевых. Однажды бизнес ...,0,1.0,семья сергей и ольга князев однажды бизнес сер...
3,Кружева,2014,"о талантливой девушке Вере, которая работает ...",0,2.0,о талантливый девушка вера который работать шв...
4,Пассажир из Сан-Франциско,2019,Калифорния. 1 сентября 2004 года. По шоссе вдо...,1,1.0,калифорния сентябрь год по шоссе вдоль побереж...
5,Красавец и чудовище,2014,В 2016 году Вероника Пляшкевич и Алексей Анище...,1,1.0,в год вероника пляшкевич и алексей анищенко сы...
...,...,...,...,...,...,...
573,Дом для куклы,2016,"\nГлавная героиня сериала, девушка по имени Ди...",0,NaN,главный героиня сериал девушка по имя дина нич...
574,Прилетит вдруг волшебник,2008,Вера - красивая молодая женщина никак не может...,0,NaN,вера красивый молодой женщина никак не мочь ус...
575,Вопреки всему,2014,"играет молодую девушку Наташу, жизнь которой ...",0,NaN,играть молодой девушка наташа жизнь который мо...
576,Хозяйка «Белых ночей»,2011,\nМарина – хозяйка уютной гостиницы «Белые ноч...,0,NaN,марина хозяйка уютный гостиница белый ночь рас...


Наконец, удаляем стоп-слова! И составляем свой список стоп-слов, которые содержатся в описаниях фильмов, но не несут содержательного смысла для анализа, а, наоборот, могут ему навредить. Мы хоть и почистили вручную те наблюдения, где не полное описание сюжета, а краткая аннотация, однако все-таки что-то проглядели.

In [14]:
import stop_words

In [15]:
sw = stop_words.get_stop_words('russian')
sw

['а',
 'в',
 'г',
 'е',
 'ж',
 'и',
 'к',
 'м',
 'о',
 'с',
 'т',
 'у',
 'я',
 'бы',
 'во',
 'вы',
 'да',
 'до',
 'ее',
 'ей',
 'ею',
 'её',
 'же',
 'за',
 'из',
 'им',
 'их',
 'ли',
 'мы',
 'на',
 'не',
 'ни',
 'но',
 'ну',
 'нх',
 'об',
 'он',
 'от',
 'по',
 'со',
 'та',
 'те',
 'то',
 'ту',
 'ты',
 'уж',
 'без',
 'был',
 'вам',
 'вас',
 'ваш',
 'вон',
 'вот',
 'все',
 'всю',
 'вся',
 'всё',
 'где',
 'год',
 'два',
 'две',
 'дел',
 'для',
 'его',
 'ему',
 'еще',
 'ещё',
 'или',
 'ими',
 'имя',
 'как',
 'кем',
 'ком',
 'кто',
 'лет',
 'мне',
 'мог',
 'мож',
 'мои',
 'мой',
 'мор',
 'моя',
 'моё',
 'над',
 'нам',
 'нас',
 'наш',
 'нее',
 'ней',
 'нем',
 'нет',
 'нею',
 'неё',
 'них',
 'оба',
 'она',
 'они',
 'оно',
 'под',
 'пор',
 'при',
 'про',
 'раз',
 'сам',
 'сих',
 'так',
 'там',
 'тем',
 'тех',
 'том',
 'тот',
 'тою',
 'три',
 'тут',
 'уже',
 'чем',
 'что',
 'эта',
 'эти',
 'это',
 'эту',
 'алло',
 'буду',
 'будь',
 'бывь',
 'была',
 'были',
 'было',
 'быть',
 'вами',
 'ваша',
 

In [16]:
text = df.loc[145, 'description_prep']
text

'лера успешный бизнесвумен который весь время посвящать свой работа из за этот она совсем забывать о личный жизнь и даже не замечать как начинать терять самый близкий она человек любимый муж вадим что же делать к счастие для лера существовать новогодний ночь в который происходить разный чудо и быть возможность исправлять любой ошибка что быть делать героиня и смочь ли она удерживать муж изменять ли она свой жизнь после этот ночь чтобы узнавать предлагать посмотреть новогодний мелодрама ошибка любовь в наш онлайн кинотеатр она понравиться все кто любить романтический кино на тема праздник'

In [17]:
new_text = []
for word in text.split(' '):
    if word not in sw:
        new_text.append(word)
' '.join(new_text)

'лера успешный бизнесвумен посвящать свой работа забывать личный замечать начинать терять самый близкий любимый муж вадим делать счастие лера существовать новогодний ночь происходить разный чудо возможность исправлять любой ошибка делать героиня смочь удерживать муж изменять свой ночь узнавать предлагать посмотреть новогодний мелодрама ошибка любовь онлайн кинотеатр понравиться любить романтический кино тема праздник'

In [23]:
our_sw = ["онлайн", "мелодрама", "кинотеатр", "фильм", "кино", "узнавать", "смотреть", "свой"]

In [24]:
sw.extend(our_sw)

In [25]:
sw = list(set(sw))

In [26]:
def delete_sw(text):
    new_text = []
    for word in text.split(' '):
        if word not in sw:
            new_text.append(word)
    return ' '.join(new_text)

In [27]:
delete_sw(df.loc[145, 'description_prep'])

'лера успешный бизнесвумен посвящать работа забывать личный замечать начинать терять самый близкий любимый муж вадим делать счастие лера существовать новогодний ночь происходить разный чудо возможность исправлять любой ошибка делать героиня смочь удерживать муж изменять ночь предлагать посмотреть новогодний ошибка любовь понравиться любить романтический тема праздник'

In [28]:
delete_sw(df.loc[258, 'description_prep'])

'талантливый актриса ника окружать влюбленный поклонник восторженный критика привыкать слава богатство пойти сильный психологический травма ника помнить целый месяц находиться плен маньяк чудо сбегать безутешный мать умирать инфаркт дождаться возвращение девочка ника приходиться вспомнить переживать происходить новый похищение прежний почерк вместе бывший следователь петр пытаться находить маньяк отомщать прошлое предотвращать новый жертва'

In [29]:
df['description_prep'] = df['description_prep'].apply(delete_sw)
df

,title,year,description,desc_bin,province,description_prep
1,Он – дракон,2015,"В давние времена не было у людей счастья, вмес...",1,1.0,давний счастие вместо глаз слеза вместо сердце...
2,Спасти мужа,2011,семья Сергея и Ольги Князевых. Однажды бизнес ...,0,1.0,семья сергей ольга князев бизнес сергей сталки...
3,Кружева,2014,"о талантливой девушке Вере, которая работает ...",0,2.0,талантливый девушка вера работать швея фабрика...
4,Пассажир из Сан-Франциско,2019,Калифорния. 1 сентября 2004 года. По шоссе вдо...,1,1.0,калифорния сентябрь шоссе вдоль побережье мчат...
5,Красавец и чудовище,2014,В 2016 году Вероника Пляшкевич и Алексей Анище...,1,1.0,вероника пляшкевич алексей анищенко сыграть гл...
...,...,...,...,...,...,...
573,Дом для куклы,2016,"\nГлавная героиня сериала, девушка по имени Ди...",0,NaN,главный героиня сериал девушка дина ничто обде...
574,Прилетит вдруг волшебник,2008,Вера - красивая молодая женщина никак не может...,0,NaN,вера красивый молодой женщина никак устраивать...
575,Вопреки всему,2014,"играет молодую девушку Наташу, жизнь которой ...",0,NaN,играть молодой девушка наташа складываться абс...
576,Хозяйка «Белых ночей»,2011,\nМарина – хозяйка уютной гостиницы «Белые ноч...,0,NaN,марина хозяйка уютный гостиница белый ночь рас...


Создаем матрицу токенов и выставляем ограничение в 3%, чтобы убрать редкие слова и привести матрицу к квадратному виду.

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(min_df = 0.03).fit(df['description_prep'])
matrix = cvect.transform(df['description_prep'])
matrix = pd.DataFrame(matrix.toarray(), index = df.index, columns = cvect.get_feature_names())
matrix

,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,аня,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,2,0,0,0,0,0,8,0,...,3,2,0,0,1,2,0,0,0,2
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Добавляем к получившейся матрице слов столбец с кодированием

In [32]:
matrix = pd.concat([df['province'], matrix], axis = 1)
matrix

,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
1,1.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,2,2,0,0,0,0,0,8,...,3,2,0,0,1,2,0,0,0,2
5,1.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
575,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Делим на 2 таблицы: первая таблица содержит размеченные новости, на которых будет проходить обучение и тестрирование, вторая – неразмеченные. 

In [33]:
df_known = matrix[matrix['province'].notna()].copy()
df_unknown = matrix[matrix['province'].isna()].copy()
display(df_known, df_unknown)

,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
1,1.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,2,2,0,0,0,0,0,8,...,3,2,0,0,1,2,0,0,0,2
5,1.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
487,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,1.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
491,NaN,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
492,NaN,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493,NaN,0,0,0,0,0,0,0,3,0,...,0,1,1,1,0,0,0,0,1,0
494,NaN,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,1
495,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
575,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Создаем 2 новых объекта: 1 – для зависимой переменной, 2 – для независимой (токены из матрицы). Выводим токены с шагом 10 для предварительно ознакомления.

In [36]:
dep_variable_bin = 'province'
indep_variables = list(df_known.columns)[2:]
indep_variables[::10]

['автомобиль',
 'бабушка',
 'большой',
 'быстро',
 'взять',
 'внезапно',
 'воспитывать',
 'выбор',
 'выходить',
 'город',
 'даша',
 'детский',
 'должный',
 'друг',
 'жениться',
 'заводить',
 'зарабатывать',
 'знакомство',
 'извиняться',
 'итог',
 'комната',
 'лицо',
 'маленький',
 'место',
 'москва',
 'называть',
 'начало',
 'непростой',
 'новость',
 'обнаруживать',
 'одежда',
 'оставаться',
 'отказываться',
 'ошибка',
 'переставать',
 'поговорить',
 'позволять',
 'полностью',
 'порог',
 'потерять',
 'предложение',
 'придумывать',
 'провинциальный',
 'просыпаться',
 'рабочий',
 'разговор',
 'решать',
 'россия',
 'свадьба',
 'семья',
 'ситуация',
 'случай',
 'собирать',
 'состояние',
 'сразу',
 'стена',
 'супруга',
 'татьяна',
 'убегать',
 'уезжать',
 'утро',
 'ход',
 'чистый',
 'язык']

Случайным образом делим таблицу с размеченными новостями так, чтобы первая подвыборка, на которой будет обучаться модель, составляла 70%, а подвыборка для тестирования – 30% соответственно.

In [37]:
import numpy as np
np.random.seed(20)
test_data_idx = np.random.choice(df_known.index, round(len(df_known)*0.3), replace = False)
test_data = df_known.loc[test_data_idx].copy()
train_data = df_known.drop(test_data_idx).copy()
display(df_known, test_data, train_data)

,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
1,1.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,2,2,0,0,0,0,0,8,...,3,2,0,0,1,2,0,0,0,2
5,1.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
487,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,1.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
157,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
210,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
352,1.0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
346,2.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201,2.0,0,0,0,0,0,2,0,0,0,...,0,0,0,1,0,0,0,0,0,0
467,1.0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
412,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


,province,авария,автомобиль,агентство,актриса,александр,алексей,андрей,анна,антон,...,чувство,чувствовать,чудо,чужой,шанс,школа,юля,юный,являться,язык
1,1.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,2,2,0,0,0,0,0,8,...,3,2,0,0,1,2,0,0,0,2
5,1.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,1.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
486,1.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
487,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Переходим к построению предсказательных моделей. Сначала строим самое глубокое дерево с минимальным числом наблюдений в узлах.

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

classifier = DecisionTreeClassifier(min_samples_leaf = 1, 
                                   min_samples_split = 2, 
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

Смотрим на кол-во узлов и глубину.

In [39]:
classifier.tree_.node_count, classifier.tree_.max_depth 

(61, 13)

Сохряняем предсказания для обучающей и тестовой подвыборок.

In [40]:
predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

display(predictions_bin_train, predictions_bin_test)

1      1
2      1
4      1
5      1
6      1
      ..
484    1
486    1
487    1
489    1
490    1
Name: Predictions, Length: 343, dtype: int32

157    1
210    1
277    2
352    1
346    2
      ..
401    2
201    2
467    1
412    1
46     1
Name: Predictions, Length: 147, dtype: int32

Переходим к оценке результатов предсказания модели. В первую очередь, обращаем внимание на категорию, которая является ненаполненной и которая нас интересует больше всего – 2 (упоминание деревни или провинциального происхождения героя). Данная модель достаточно хорошо предсказывает нашу категорию (f1 = 0.67). Средние значения по модели также довольно высокие.

In [41]:
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

              precision    recall  f1-score   support

           1       0.87      0.88      0.87       106
           2       0.68      0.66      0.67        41

    accuracy                           0.82       147
   macro avg       0.77      0.77      0.77       147
weighted avg       0.82      0.82      0.82       147



Выводим токены, которые позволяют лучше всего предсказывать категории. В роли самых важных выступают: городок, деревня, столица. Многие слова из данного списка, действительно, могут судить о провинциальности героев. 

In [44]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:35]

городок           0.172300
деревня           0.166169
столица           0.133996
москва            0.055921
попадать          0.054244
приходить         0.033814
счастливый        0.032212
провинциальный    0.028323
парень            0.026772
видеть            0.026298
приходиться       0.024040
воспитывать       0.021464
обычный           0.019405
картина           0.017095
переезжать        0.015505
совершенно        0.015298
режиссер          0.015225
приезжать         0.015181
появляться        0.014958
начальник         0.014794
удар              0.014669
признаваться      0.012211
постоянно         0.012020
образ             0.012020
помогать          0.010684
просить           0.008013
казаться          0.008013
старший           0.007786
николай           0.007640
соглашаться       0.003930
крупный           0.000000
купить            0.000000
красивый          0.000000
язык              0.000000
личный            0.000000
dtype: float64

Сохраняем вероятности текстов попасть в ту или иную категорию на проверочной подвыборке. Выводим кол-во размеченных, на основании алгоритма, описаний фильмов, попадающих во 2 категорию (упоминание о провинциальности), и, действительно, тексты являются таковыми.

In [45]:
probabilities1 = classifier.predict_proba(df_unknown[indep_variables])
probabilities1 = pd.DataFrame(probabilities1, index = df_unknown.index, columns = ['ДеревоГл1', 'ДеревоГл2'])
probabilities1

,ДеревоГл1,ДеревоГл2
491,1.0,0.0
492,1.0,0.0
493,0.0,1.0
494,1.0,0.0
495,1.0,0.0
...,...,...
573,1.0,0.0
574,1.0,0.0
575,0.0,1.0
576,1.0,0.0


In [46]:
probabilities1['ДеревоГл2'].sum()

21.0

In [47]:
idx_high_proba = probabilities1['ДеревоГл2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

35-летняя мать-одиночка Люба работает секретарем в сельсовете. Соседи ее уважают, советуются по бытовым вопросам. Неожиданно возвращаются два человека из ее прошлого, и все идет кувырком. Первый – Виктор. Он много лет ухаживал за Любой и вернулся в деревню в качестве участкового. Сразу взял на себя инициативу, не давая продыху предмету обожания. Второй – Николай. Первая любовь, от него Люба родила сына, но по неведомым причинам пара разошлась, и мужчина уехал за границу. Теперь, когда оба приехали, Люба находится в центре внимания, все сплетничают, у каждого свои доводы и фантазии. Николай знакомится с сыном Степаном, радуется отцовству и находит с парнем общий язык, проводит много времени. Виктор тоже не отступает, считает Любу своей женщиной и готов бороться за нее до победного конца. Чувства Любы к Николаю не угасли и даже окрепли с возрастом. Любовный треугольник с острыми гранями делает больно всем его участникам. Все перемешалось вокруг… Как героиня выпутается из ситуации и кого 

Нами были построены деревья с помощью самостоятельного подбора параметров. Лучшим оказалось дерево с минимальным кол-вом наблюдений в детском узле, равном 10. f1 повысился на 3 %. Все выведенные нами описания фильмов, действительно содержат упоминания о провинциальном происхождении героев. 

In [59]:
classifier = DecisionTreeClassifier(min_samples_leaf = 10,
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

probabilities2 = classifier.predict_proba(df_unknown[indep_variables])
probabilities2 = pd.DataFrame(probabilities2, index = df_unknown.index, columns = ['Дерево1', 'Дерево2'])

idx_high_proba = probabilities2['Дерево2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       0.93      0.95      0.94       261
           2       0.82      0.78      0.80        82

    accuracy                           0.91       343
   macro avg       0.88      0.86      0.87       343
weighted avg       0.91      0.91      0.91       343

test data
              precision    recall  f1-score   support

           1       0.87      0.92      0.89       106
           2       0.75      0.66      0.70        41

    accuracy                           0.84       147
   macro avg       0.81      0.79      0.80       147
weighted avg       0.84      0.84      0.84       147

История о певце Андрее Ильине, который стал знаменитостью. Некогда он жил в простом провинциальном городке и не мог даже мечтать о таком успехе, но фортуна повернулась к нему лицом. Этот человек давно грезил о том, чтобы стать любимцем публики, чего и достиг. Спустя некоторое время к мужчине приезжает из детского дома 11-ле

Значимых токенов стало меньше, однако большинство из них на уровне логики являются важными критериями отнесения описаний фильмов ко 2 категории.

In [62]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:15]

городок           0.267470
деревня           0.257952
столица           0.208008
москва            0.086808
приходить         0.073053
прошлое           0.029297
женщина           0.022806
город             0.022492
обещать           0.017273
провинциальный    0.008481
молодой           0.002352
марина            0.002108
детский           0.001900
лиза              0.000000
лицо              0.000000
dtype: float64

Чтобы подобрать лучшую модель, мы воспользовались кросс-валидацией. В ее результе построенная модель обладает такой же предсказательной силой, как и предыдущая. Хотя последнее описание фильма, отнесенное алгоритмом ко 2 категории, всё-таки к ней относиться не должно. Значимых токенов стало еще меньше. Отпало слово "провинциальный".

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
%%time

params = {'max_depth': list(range(1,30)),
         'min_samples_leaf': list(range(1,40))}

score = 'f1_macro'
model = DecisionTreeClassifier(random_state = 0)

cv = GridSearchCV(model, params, score, cv = 4)
cv.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

Wall time: 1min 1s


GridSearchCV(cv=4, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15

In [65]:
cv.best_params_,cv.best_score_

({'max_depth': 5, 'min_samples_leaf': 11}, 0.8308362506510178)

In [66]:
classifier = DecisionTreeClassifier(min_samples_leaf = 11,
                                   max_depth = 5,
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities3 = classifier.predict_proba(df_unknown[indep_variables])
probabilities3 = pd.DataFrame(probabilities3, index = df_unknown.index, columns = ['ДеревоКВ1', 'ДеревоКВ2'])

idx_high_proba = probabilities3['ДеревоКВ2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       0.93      0.95      0.94       261
           2       0.82      0.78      0.80        82

    accuracy                           0.91       343
   macro avg       0.88      0.86      0.87       343
weighted avg       0.91      0.91      0.91       343

test data
              precision    recall  f1-score   support

           1       0.87      0.92      0.89       106
           2       0.75      0.66      0.70        41

    accuracy                           0.84       147
   macro avg       0.81      0.79      0.80       147
weighted avg       0.84      0.84      0.84       147

 деловую женщину Дарью Кирилловну. Сбежав из нищей деревни в Москву, скромница Даша начала пробивать себе путь к богатству. Вскоре она стала жесткой и своенравной женщиной – главой модного архитектурного бюро. Сейчас у нее огромный загородный дом и муж, который является партнером в ее фирме. Однако всего этого она лишается 

In [69]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:10]

городок        0.291360
деревня        0.280992
столица        0.226588
москва         0.094562
приходить      0.052894
город          0.029630
место          0.022742
становиться    0.001232
язык           0.000000
любимый        0.000000
dtype: float64

Переходим к построению ансамбля деревьев. Сначала реализуем модель случайного леса. Модель с настройками по умолчанию уступает всем предыдущим (с т.з. предсказания категории 2). Кроме того, стоит сказать, что вообще модель случайного леса оказалась хуже, чем любая модель дерева решений. Достаточно низкий recall, заметна переобученность. Но в целом алгоритм правильно разметил первые 5 новостей. Список важных слов достаточно объемный и содержит, на наш взгляд, не несущие смысла для категоризации токены. 

In [70]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [80]:
rf_classifier = RandomForestClassifier(n_estimators=100,
                                      max_depth=None,
                                      random_state=0)
rf_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = rf_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = rf_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.79      0.98      0.87       106
           2       0.87      0.32      0.46        41

    accuracy                           0.80       147
   macro avg       0.83      0.65      0.67       147
weighted avg       0.81      0.80      0.76       147



In [85]:
rf_classifier = RandomForestClassifier(n_estimators=200,
                                      max_depth=30,
                                      random_state=0)
rf_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = rf_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = rf_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities4 = rf_classifier.predict_proba(df_unknown[indep_variables])
probabilities4 = pd.DataFrame(probabilities4, index = df_unknown.index, columns = ['Лес1', 'Лес2'])

idx_high_proba = probabilities4['Лес2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.81      0.99      0.89       106
           2       0.94      0.39      0.55        41

    accuracy                           0.82       147
   macro avg       0.87      0.69      0.72       147
weighted avg       0.84      0.82      0.80       147

 провинциальной почтальонши, которая, помимо своей основной работы, занимается еще и сводничеством. почтальона Людмилу Пичугину. Развозя почту, Люда ездит по соседним селам и деревням и там присматривает женихов для своих подруг и просто знакомых одиноких девушек. Героиня сводит одну счастливую пару за другой, но сама продо

In [86]:
feat_imp = rf_classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:50]

деревня           0.057480
городок           0.042827
столица           0.042427
провинциальный    0.033464
москва            0.026695
город             0.020684
обычный           0.016098
приезжать         0.010326
жить              0.008664
бизнесмен         0.007875
местный           0.007644
уезжать           0.006892
петербург         0.006256
прошлое           0.005626
поступать         0.005576
школа             0.005127
родной            0.004829
ребенок           0.004347
институт          0.004326
девушка           0.004320
возникать         0.004312
женщина           0.004311
несмотря          0.004303
родитель          0.004089
серьезный         0.003976
взрослый          0.003975
развод            0.003858
ради              0.003819
семья             0.003754
большой           0.003746
находить          0.003744
помогать          0.003722
мужчина           0.003684
парень            0.003595
соглашаться       0.003590
молодой           0.003531
отец              0.003503
д

Переходим ко второму типу ансамблей. Реализуем бустинг. Результат подбора параметров вручную является наилучшим: она лучше всех предыдущих предсказывает важную для нас и менее наполненную категорию (2 модель, f1 = 0.72). Вероятно, она является переобученной. Кроме того, текст 4, отнесенный ко второй категории, является спорным случаем. Автоматический поиск оптимальных настроек для бустинга, в свою очередь, подобрал параметры хуже, чем мы самостоятельно. 

In [87]:
gb_classifier = GradientBoostingClassifier(n_estimators=100,
                                      max_depth=None,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.87      0.87      0.87       106
           2       0.66      0.66      0.66        41

    accuracy                           0.81       147
   macro avg       0.76      0.76      0.76       147
weighted avg       0.81      0.81      0.81       147



In [104]:
gb_classifier = GradientBoostingClassifier(n_estimators=100,
                                      max_depth=14,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities5 = gb_classifier.predict_proba(df_unknown[indep_variables])
probabilities5 = pd.DataFrame(probabilities5, index = df_unknown.index, columns = ['Бустинг1', 'Бустинг2'])

idx_high_proba = probabilities5['Бустинг2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')


train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.89      0.90      0.89       106
           2       0.72      0.71      0.72        41

    accuracy                           0.84       147
   macro avg       0.81      0.80      0.80       147
weighted avg       0.84      0.84      0.84       147

История о певце Андрее Ильине, который стал знаменитостью. Некогда он жил в простом провинциальном городке и не мог даже мечтать о таком успехе, но фортуна повернулась к нему лицом. Этот человек давно грезил о том, чтобы стать любимцем публики, чего и достиг. Спустя некоторое время к мужчине приезжает из детского дома 11-ле

In [105]:
feat_imp = gb_classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:50]

городок           0.172336
деревня           0.166203
столица           0.134025
москва            0.055927
приходить         0.033977
провинциальный    0.028329
приезжать         0.023729
попадать          0.022593
обычный           0.021605
приходиться       0.017097
счастливый        0.016889
картина           0.016652
видеть            0.016171
замуж             0.015333
совершенно        0.015197
удар              0.015124
хватать           0.012913
большой           0.010553
убивать           0.010295
ничто             0.009692
появляться        0.009304
лежать            0.009061
воспитывать       0.008224
парень            0.008007
петербург         0.007904
агентство         0.006309
город             0.005093
переезжать        0.005023
влиятельный       0.004965
старший           0.004953
мама              0.004568
просить           0.004521
николай           0.004415
надежда           0.004114
одиночество       0.003876
постоянно         0.003772
девушка           0.003751
и

In [106]:
import warnings
warnings.filterwarnings('ignore')

In [107]:
%%time

params = {'max_depth': list(range(8,21, 2)),
         'n_estimators': list(range(50,151,10))}

score = 'f1'
model = GradientBoostingClassifier(random_state = 0)

cv = GridSearchCV(model, params, score, cv = 4)
cv.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

Wall time: 4min 8s


GridSearchCV(cv=4, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [108]:
cv.best_params_,cv.best_score_

({'max_depth': 10, 'n_estimators': 80}, 0.9102419248295577)

In [109]:
gb_classifier = GradientBoostingClassifier(n_estimators=80,
                                      max_depth=10,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))



train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       261
           2       1.00      1.00      1.00        82

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.86      0.90      0.88       106
           2       0.69      0.61      0.65        41

    accuracy                           0.82       147
   macro avg       0.78      0.75      0.76       147
weighted avg       0.81      0.82      0.81       147



Создаем общую таблицу для всех успешных моделей с вероятностями попадания в ту или иную категорию и добавим тексты описаний фильмов.

In [110]:
probabilities_full = pd.concat([probabilities1, probabilities2, probabilities3, probabilities4, probabilities5], axis = 1)
probabilities_full

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2
491,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.950000,0.050000,0.999857,0.000143
492,1.0,0.0,0.600000,0.400000,0.959641,0.040359,0.845000,0.155000,0.999872,0.000128
493,0.0,1.0,0.727273,0.272727,0.959641,0.040359,0.555000,0.445000,0.999447,0.000553
494,1.0,0.0,0.727273,0.272727,0.959641,0.040359,0.560000,0.440000,0.207962,0.792038
495,1.0,0.0,0.625000,0.375000,0.750000,0.250000,0.785000,0.215000,0.999811,0.000189
...,...,...,...,...,...,...,...,...,...,...
573,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.939942,0.060058,0.999858,0.000142
574,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.845000,0.155000,0.999857,0.000143
575,0.0,1.0,0.352941,0.647059,0.352941,0.647059,0.670000,0.330000,0.000994,0.999006
576,1.0,0.0,0.900000,0.100000,0.959641,0.040359,0.835000,0.165000,0.039033,0.960967


In [111]:
probabilities_full['Текст'] = df['description']
probabilities_full

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2,Текст
491,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.950000,0.050000,0.999857,0.000143,\n\nСемейная пара Денис и Арина Ковровы двадца...
492,1.0,0.0,0.600000,0.400000,0.959641,0.040359,0.845000,0.155000,0.999872,0.000128,По трагическому стечению обстоятельств девушка...
493,0.0,1.0,0.727273,0.272727,0.959641,0.040359,0.555000,0.445000,0.999447,0.000553,Наталья (Туся) находится в больничной палате. ...
494,1.0,0.0,0.727273,0.272727,0.959641,0.040359,0.560000,0.440000,0.207962,0.792038,"На границе России и Грузии, неподалеку от осет..."
495,1.0,0.0,0.625000,0.375000,0.750000,0.250000,0.785000,0.215000,0.999811,0.000189,"Оксана и Игорь – молодая счастливая пара, кото..."
...,...,...,...,...,...,...,...,...,...,...,...
573,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.939942,0.060058,0.999858,0.000142,"\nГлавная героиня сериала, девушка по имени Ди..."
574,1.0,0.0,1.000000,0.000000,0.959641,0.040359,0.845000,0.155000,0.999857,0.000143,Вера - красивая молодая женщина никак не может...
575,0.0,1.0,0.352941,0.647059,0.352941,0.647059,0.670000,0.330000,0.000994,0.999006,"играет молодую девушку Наташу, жизнь которой ..."
576,1.0,0.0,0.900000,0.100000,0.959641,0.040359,0.835000,0.165000,0.039033,0.960967,\nМарина – хозяйка уютной гостиницы «Белые ноч...


В качестве лучших алгоритмов у нас выступили бустинг (f1 = 0.72) и 2 дерева (f1 = 0.70). Для начала сравним предсказания двух деревьев, которые различаются больше всего.

In [113]:
probabilities_full['Вычисления_дер'] = abs(probabilities_full['ДеревоКВ2']-probabilities_full['Дерево2'])
probabilities_full = probabilities_full.sort_values('Вычисления_дер', ascending=False)
probabilities_full[:15]

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2,Текст,Вычисления_дер
531,0.0,1.0,0.400000,0.600000,0.000000,1.000000,0.425000,0.575000,0.000250,0.999750,деловую женщину Дарью Кирилловну. Сбежав из н...,0.400000
542,0.0,1.0,0.000000,1.000000,0.363636,0.636364,0.545000,0.455000,0.000366,0.999634,Ольга Краснова – успешный художник-модельер. Н...,0.363636
492,1.0,0.0,0.600000,0.400000,0.959641,0.040359,0.845000,0.155000,0.999872,0.000128,По трагическому стечению обстоятельств девушка...,0.359641
544,1.0,0.0,0.600000,0.400000,0.959641,0.040359,0.879942,0.120058,0.999872,0.000128,Супруги Алексей и Марина выдали замуж свою еди...,0.359641
572,1.0,0.0,0.600000,0.400000,0.959641,0.040359,0.805000,0.195000,0.999872,0.000128,Матушка Екатерина в прошлом не смогла пережить...,0.359641
511,1.0,0.0,1.000000,0.000000,0.750000,0.250000,0.735000,0.265000,0.999783,0.000217,Большой город. Рассвет. Молодой мужчина по име...,0.250000
570,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.565000,0.435000,0.091771,0.908229,"Англичанин Эллиот молод и хорош собой, но выле...",0.250000
518,1.0,0.0,1.000000,0.000000,0.750000,0.250000,0.605000,0.395000,0.999844,0.000156,В начале фильма идет рассказ о популярном в ст...,0.250000
514,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.540000,0.460000,0.000842,0.999158,В провинциальном городе проходит экономический...,0.250000
508,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.655000,0.345000,0.178816,0.821184,Недалекое будущее. Пожилой фотограф Никита Пир...,0.250000


Предсказания деревьев одинаковые. Выбираем дерево, построенное с помощью кросс-валидации, и сравниваем различающиеся с бустингом предсказания (таких 7).

In [115]:
probabilities_full['Вычисления_буст'] = abs(probabilities_full['ДеревоКВ2']-probabilities_full['Бустинг2'])
probabilities_full = probabilities_full.sort_values('Вычисления_буст', ascending=False)
probabilities_full[:10]

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2,Текст,Вычисления_дер,Вычисления_буст
566,0.0,1.0,0.900000,0.100000,0.959641,0.040359,0.500,0.500,0.000684,0.999316,"\nТихий посёлок с красивой природой, где люди ...",0.059641,0.958957
576,1.0,0.0,0.900000,0.100000,0.959641,0.040359,0.835,0.165,0.039033,0.960967,\nМарина – хозяйка уютной гостиницы «Белые ноч...,0.059641,0.920608
494,1.0,0.0,0.727273,0.272727,0.959641,0.040359,0.560,0.440,0.207962,0.792038,"На границе России и Грузии, неподалеку от осет...",0.232369,0.751679
514,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.540,0.460,0.000842,0.999158,В провинциальном городе проходит экономический...,0.250000,0.749158
552,1.0,0.0,0.625000,0.375000,0.750000,0.250000,0.720,0.280,0.023861,0.976139,Лиза мечтает побывать в Венеции. Вместе с ней ...,0.125000,0.726139
570,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.565,0.435,0.091771,0.908229,"Англичанин Эллиот молод и хорош собой, но выле...",0.250000,0.658229
508,0.0,1.0,1.000000,0.000000,0.750000,0.250000,0.655,0.345,0.178816,0.821184,Недалекое будущее. Пожилой фотограф Никита Пир...,0.250000,0.571184
542,0.0,1.0,0.000000,1.000000,0.363636,0.636364,0.545,0.455,0.000366,0.999634,Ольга Краснова – успешный художник-модельер. Н...,0.363636,0.363271
559,0.0,1.0,0.352941,0.647059,0.352941,0.647059,0.450,0.550,0.000438,0.999562,"\nМолодая, совсем еще не знающая жизни Эля Най...",0.000000,0.352504
575,0.0,1.0,0.352941,0.647059,0.352941,0.647059,0.670,0.330,0.000994,0.999006,"играет молодую девушку Наташу, жизнь которой ...",0.000000,0.351947


В первом случае бустинг правильно определил попадание описания текста во вторую категорию.

In [116]:
probabilities_full['Текст'][566]

'\nТихий посёлок с красивой природой, где люди не привыкли к суете, нервозности и ярким событиям. Таня – вчерашняя школьница, мягкая и мечтательная, которая втайне пишет рассказы, погружаясь в мир фантазий. Но её реальная жизнь заурядна и скучна: она получает низкий оклад в больнице и воспитывает брата и сестру, заменяя им родителей. Бесконечную рутину прерывает знакомство с молодым врачом Тимуром. Впервые Таня чувствует, что её жизнь обретает краски.\nЭта жизненная мелодрама от молодого режиссёра-дебютантки Александры Ерофеевой рисует трогательный портрет провинциальной жизни, обычных людей и искренних чувств. Фильм рассказывает о личном счастье и личных трагедиях, которые случаются всегда и повсеместно, а лейтмотивом к происходящему звучат стихи Иосифа Бродского.\n'

Здесь ошибся бустинг – всё-таки в данном случае нет указаний ни на деревню, село, ни на провинциальное происхождение кого-либо из героев.

In [117]:
probabilities_full['Текст'][576]

'\nМарина – хозяйка уютной гостиницы «Белые ночи», расположенной в самом центре Петербурга. Она молода и привлекательна, окружена мужским вниманием, но никому не отвечает взаимностью. Все дело в том, что десять лет назад Марине разбили сердце, и с тех пор она решила отказаться от романтических отношений и использовать мужчин «по их прямому назначению», например, чтобы сделать в гостинице капремонт или срочно оформить необходимые документы. \nНо однажды в гостинице Марины появляется постоялец, над которым не властны ее чары. Это капитан первого ранга Константин Соломатин. Он не просто демонстрирует полное равнодушие к Марине, но и буквально обвиняет ее в манипулировании мужчинами. Марина не намерена терпеть подобные оскорбления и решает отомстить капитану: она непременно влюбит его в себя, а затем заставит страдать от неразделенной любви.\n'

В данном случае речь идет о селе, хоть и о грузинском, а потому бустинг правильно отнес описание фильма ко второй категории.

In [118]:
probabilities_full['Текст'][494]

'На границе России и Грузии, неподалеку от осетинского села Толи и грузинского села Тэли в горах пасут овец два пастуха: грузин Валико и осетин Шалико. Шалико говорит Валико, что у него два барана сбежали в Грузию: и что их так за границу тянет? Это какие, с большими курдюками? Да. Валико: нет, я их не видел. К пастухам подходит лейтенант пограничной грузинской службы Гурам Антелава. Он рассказывает пастухам, что жена обещала ему осенью родить сына, но до этого обманывала уже в третий раз.Священник отец Николай, сидя на крыше старинной церкви, оглядывает окрестности в подзорную трубу. Он видит, что на горной дороге останавливается рейсовый автобус. Оттуда выходит девушка, идет по направлению к Толи. Отец Николай обращается к своему псу: смотри, Гамлет, какая Сатиник красавица стала!На мосту через пограничную речку стоят Гурам и сержант пограничных войск РФ Максим Лапшин. Они слышат выстрел в горах, начинают спорить. Гурам утверждает, что это карабин, а Максим настаивает на том, что про

Здесь снова бустинг определил правильно, тк речь в тесте идет о провинциальном городе.

In [119]:
probabilities_full['Текст'][514]

'В провинциальном городе проходит экономический форум, но котором собираются видные бизнесмены и изобретатели со всего мира. В одном из отелей, куда заселяют гостей, горничной работает красивая девушка Надежда. Администрация мэрии заранее предупреждает владелицу гостиницы о важности мероприятия. Главный гость – ученый и предприниматель Олег Анатольевич Ворохов, которому нужно уделить особое внимание. \nВладелица этого отеля Изабелла рассказывает дочери Маше о своих проблемах. Оказывается, взятый на развитие бизнеса кредит потрачен, а накопленные проценты уже превышают сумму займа. Чтобы выпутаться из ситуации, женщина задумала выдать замуж Машу за бизнесмена Олега и получить его помощь. Она всячески пытается угодить гостю, но делает только хуже: навязчивый сервис тяготит Олега, а в его люксе не оказывается интернета.\nВ гостинице бизнесмен случайно знакомится с горничной Надей, и они начинают общаться. Богатый холостяк очарован красотой и искренностью провинциалки. Между ними вспыхиваю

В этом случае мы склонны считать, что бустинг ошибся, так как про место проживания героев ничего не сказано.

In [120]:
probabilities_full['Текст'][552]

'Лиза мечтает побывать в Венеции. Вместе с ней в этот волшебный город хочет поехать и ее возлюбленный Саша. Выполнить задуманное они планируют сразу же поле выпускных экзаменов в университете. Парень изо всех сил старается приблизиться к поставленной цели. Чтобы заработать денег на поездку, он соглашается на любую работу. В его семье Лизу воспринимают как будущую невестку. Но у ее родителей совсем другие планы. Ведь они лучше знают, что нужно их дочери. Предприимчивые «предки» пытаются с помощью влиятельного друга организовать не только перспективную вакансию для своей дочери, а еще и более удачную партию, чем профессорский сын, у которого недавно умер отец. Влюбленные расстаются на целых десять лет. В университете состоится встреча выпускников. Герои приезжают, чтобы вспомнить студенческие годы. Случайная встреча. Он успешный адвокат, у нее растет дочь. Но старое, казалось бы, забытое чувство вспыхивает с новой силой. Лиза принимает решение уйти из семьи к любимому. Ее муж не может эт

Здесь также ошибается бустинг – вряд ли Англия является провинцией)

In [121]:
probabilities_full['Текст'][570]

'Англичанин Эллиот молод и хорош собой, но вылетел уже из двух университетов и сменил множество работ. К тому же молодой человек не ладит с успешным в карьере отцом, который упрекает сына в разгильдяйстве. Однажды друг рассказывает ему об интересной работе, которую можно найти в интернете. Правда, трудиться придется в другой стране. Эллиота привлекает вакансия репетитора по английскому языку в Санкт-Петербурге, он решает ехать. По дороге в аэропорт папа признаётся ему, что в Питере у него есть взрослый сын от другой женщины, о котором никто не знает, и хочет познакомить братьев. Но парень от шока и обиды посылает отца «куда подальше» и уезжает.Прибыв в город белых ночей, он поселяется в дешёвом хостеле. На следующий день Эллиот неудачно проходит собеседование, и у него не остаётся денег даже на еду. От безысходности он звонит Фёдору, сыну своего отца, и просит о встрече. Русский брат принимает его радушно, сам он учится на режиссёра и актёра, поэтому график у него плотный.Британец знак

Наверное, Нижний Новгород является провинцией, по сравнению с Питером, поэтому в этом случае предсказать категорию лучше получилось у бустинга. 

In [122]:
probabilities_full['Текст'][508]

'Недалекое будущее. Пожилой фотограф Никита Пирогов рассказывает историю своей жизни молодой девушке Анне, которая пришла в его студию. Все началось в 1995 году, когда еще молодой Никита только начинал заниматься фотографией и однажды утром познакомился с девушкой по имени Наташа. Она забралась на перила высокого моста, и Никита подумал, что она собралась прыгать. Однако Наташа оказалась очень веселой и жизнерадостной девушкой. Молодые люди прогулялись по Нижнему Новгороду, и зашли в Кремль. Никита рассказал Наташе о том, как в древности в стены крепости замуровывали прекрасных девиц. Также там он познакомил ее со своими знакомыми – Юрой и Андреем. Друзья рассказали другую историю – о девушке, которая одним коромыслом победила целый отряд казанского хана.Никита привел Наташу к часозвоне и рассказал, как она появилась на этом месте. Дело обстояло так. В 1913 году в городе ждали Николая II на открытие нового здания госбанка. Все было подготовлено на высшем уровне, но была одна проблема –

In [123]:
probabilities3['ДеревоКВ2'].sum()

19.042533963003496

In [124]:
probabilities5['Бустинг2'].sum()

23.46473599912473

На наш взгляд, ошибка первого рода важнее – мы не хотим, чтобы фильмы, не содержащие упоминание о деревне, провинциальном происхождении, попадали во вторую категорию, поэтому нами было принято решение выделить дерево, построенное с помощью кросс-валидации, как лучшее. 
В целом, стоит заметить, что почти все модели дали на такой небольшой подвыборке достаточно хороший результат, чего мы даже не ожидали изначально.